## Update and install packages

In [3]:
# Update and install display packages and stable baseline 3
!pip install -r ../requirements.txt
!pip install ipywidgets


  Using cached fastapi-0.115.4-py3-none-any.whl (94 kB)
  Using cached gym-0.26.2-py3-none-any.whl
  Using cached stable_baselines3-2.4.0a11-py3-none-any.whl (183 kB)
  Using cached starlette-0.41.2-py3-none-any.whl (73 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Using cached fastapi_cli-0.0.5-py3-none-any.whl (9.5 kB)
  Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
  Using cached python_multipart-0.0.17-py3-none-any.whl (24 kB)
  Using cached email_validator-2.2.0-py3-none-any.whl (33 kB)
  Using cached uvicorn-0.32.0-py3-none-any.whl (63 kB)
  Using cached gym_notices-0.0.8-py3-none-any.whl (3.0 kB)
  Using cached gymnasium-1.0.0-py3-none-any.whl (958 kB)
  Using cached pygame-2.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (14.0 MB)
  Using cached tensorboard-2.18.0-py3-none-any.whl (5.5 MB)
  Using cached ale_py-0.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Using cached dnspython-2.7.0-py3-none-any.whl 

## Import Libraries

In [ ]:
# Import rquired libraries and modules
import gymnasium as gym
from gymnasium import spaces
from gymnasium import Space
import stable_baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import torch
from pathlib import Path
from typing import List
import random
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

import sys
import os

# Add the /app/src directory to the Python path
sys.path.append(os.path.abspath('../app/src'))


# Now try importing again
from spark.data import loader
from spark.data.models import Customer, Product, Category, Interaction, InteractionType
from spark import utils


In [ ]:
# Check gym and stable baseline 3 versions
print(f"{gym.__version__=}")
print(f"{stable_baselines3.__version__=}")


## Settings

In [ ]:
# name of the environment to run
env_name = 'Spark'
env_prefix = 'spark'

# output directoies
base_dir = '.'
output_dir = os.path.join(base_dir, 'output')
env_dir =  os.path.join(output_dir, env_prefix)
logs_dir = os.path.join(env_dir, 'logs')
models_dir = os.path.join(env_dir, 'models')
videos_dir = os.path.join(env_dir, 'videos')

os.makedirs(logs_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)
os.makedirs(videos_dir, exist_ok=True)

# tensorboard name for algorithm logs
tb_log_name = 'PPO'
model_prefix = 'ppo'
model_name_final = f"{model_prefix}_model_final"

print(logs_dir)
print(models_dir)
print(videos_dir)

## Callbacks and Directory Setup

In [ ]:
# callback for saving model at regular intervals
class SaveOnIntervalCallback(BaseCallback):
    def __init__(self, save_interval: int, save_path: str, verbose=1):
        super().__init__(verbose)
        self.save_interval = save_interval
        self.save_path = save_path

    def _on_step(self) -> bool:
        # Save the model every 'save_interval' steps
        if self.num_timesteps % self.save_interval == 0:
            save_file = os.path.join(self.save_path, f'{model_prefix}_model_{self.num_timesteps}')
            self.model.save(save_file)
            if self.verbose > 0:
                print(f'Saving model to {save_file}.zip')
        return True

## Custom environment

In [ ]:

class RecommendationEnv(gym.Env):
    def __init__(self, users:List[Customer], products:List[Product], categories:List[Category], top_k:int):
        super().__init__()
        
        self.users = users                  # list of users as states
        self.products = products            # products as actions, potential recommendations
        self.categories = categories
        self.top_k = top_k                  # number of recommendations
        self.user_idx = 0                   # index of users list, not user_id
        self.current_step = 0               # step is also the interactions list index
        
        for user in users:
            user.views = np.zeros(len(products), dtype=np.int16)
            user.likes = np.zeros(len(products), dtype=np.int16)
            user.buys = np.zeros(len(products), dtype=np.int16)
            user.ratings = np.zeros(len(products), dtype=np.int16)
            for interaction in user.interactions:
                i_type = interaction.type.value    
                product_idx = interaction.product_idx  
                if i_type == InteractionType.VIEW.value:
                    user.views[product_idx] += 1
                elif i_type == InteractionType.LIKE.value:
                    user.likes[product_idx] += 1
                elif i_type == InteractionType.BUY.value:
                    user.buys[product_idx] += 1
                elif i_type == InteractionType.RATE.value:
                    user.ratings[product_idx] = interaction.value
                    
        # define action space to return top_k recommended product ids.    
        self.action_space = spaces.MultiDiscrete([len(products)] * top_k) 
        
        # number of customers as states
        # states are derived from customer profiles and interactioms
        # Users list will keep track of unique users
        # States include subset of features including product, interaction, ratings, and time in one-hot-encoding format
        # States exclude user_ids for policy network generalisation. But internal users list will be used as reference        
        self.observation_space = spaces.Dict({
            'pref_prod': spaces.Box(low=0, high=1, shape=(len(self.products),), dtype=np.float32),
            'pref_cat': spaces.Box(low=0, high=1, shape=(len(self.categories),), dtype=np.float32),
            'buys': spaces.Box(low=0, high=1000, shape=(len(self.products),), dtype=np.int16),
            'views': spaces.Box(low=0, high=1000, shape=(len(self.products),), dtype=np.int16),
            'likes': spaces.Box(low=0, high=1000, shape=(len(self.products),), dtype=np.int16),
            'ratings': spaces.Box(low=0, high=5, shape=(len(self.products),), dtype=np.uint8),
            'product': spaces.Box(low=0, high=1, shape=(len(self.products),), dtype=np.uint8),
            'interaction': spaces.Box(low=0, high=1, shape=(len(list(InteractionType)),), dtype=np.uint8),
            'rating': spaces.Discrete(6)
            }) 
        
        
    def reset(self, seed=None, options=None):
        # Call the parent class's reset method to handle seeding
        super().reset(seed=seed)
        
        self.user_idx = np.random.randint(len(self.users)) # may run throught users one by one
        user = self.users[self.user_idx]
        self.current_step = 0
        return self._get_observation(user), {}# get current user features as states

    def step(self, rec_products, interaction:Interaction=None):
        """ randomly interacting with product to mimick real user unpredictable behavious """
        self.current_step += 1
        
        # interaction passed in fro
        if interaction:    
            self.user_idx = interaction.customer_idx
        else:       
            # simulate selected recommended product and interaction
            interaction = self._simulate_interaction(rec_products) # generate random interaction
        
        reward = 0
        done = False
        
        # reward function if the recommended product is clicked
        if interaction.product_idx in rec_products:
            if interaction.type == InteractionType.NONE:
                reward = -1 # no interaction, customers not interested in recommendations
            elif interaction.type ==  InteractionType.VIEW:
                reward = 3
            elif interaction.type ==  InteractionType.LIKE:
                reward = 10
            elif interaction.type ==  InteractionType.BUY:
                reward = 30
            elif interaction.type ==  InteractionType.RATE:            
                reward = (interaction.value - 2)  * 3 # rating of 1 is negative
        
        done = interaction.type ==  InteractionType.SESSION_CLOSE
        
        interaction_info = {
                'interaction': {
                    'idx': -1,
                    'timestamp': interaction.timestamp.strftime("%Y-%m-%d %H:%M:%S"),
                    'customer_id': interaction.customer_idx,
                    'product_id': interaction.product_idx,
                    'interaction': interaction.type.value,
                    'rating': interaction.value
                }
            }
        
        user = self.users[self.user_idx]
        
        return self._update_observation(user, interaction), reward, done, False, interaction_info

    def _update_observation(self, user:Customer, interaction:Interaction):            
        if interaction.type == InteractionType.VIEW:
            user.views[interaction.product_idx] += 1
        elif interaction.type == InteractionType.LIKE:
            user.likes[interaction.product_idx] += 1
        elif interaction.type == InteractionType.BUY:
            user.buys[interaction.product_idx] += 1
        elif interaction.type == InteractionType.RATE:
            user.ratings[interaction.product_idx] = interaction.value
            
        product_obs = np.zeros(len(self.products), dtype=np.int16)
        if interaction.type == InteractionType.VIEW \
            or interaction.type == InteractionType.LIKE  \
            or interaction.type == InteractionType.BUY \
            or interaction.type == InteractionType.RATE:
                product_obs = utils.one_hot_encode(interaction.product_idx, len(self.products))
        
        # update observation based on new data  
        obs = {
                'pref_prod': self._get_product_preferences(user),
                'pref_cat': self._get_category_preferences(user), 
                'buys': user.buys,
                'views': user.views,
                'likes': user.likes,
                'ratings': user.ratings,
                'product': product_obs,
                'interaction': self._get_interaction_observation(interaction),
                'rating': interaction.value if interaction.type == InteractionType.RATE else 0 
            }
        
        return obs
    
    def update_observation(self, user:Customer, interaction:Interaction):
        return self._update_observation(user, interaction)

    def _get_observation(self, user:Customer): 
        
        obs = {
                'pref_prod': self._get_product_preferences(user),
                'pref_cat': self._get_category_preferences(user), 
                'buys': user.buys,
                'views': user.views,
                'likes': user.likes,
                'ratings': user.ratings,
                'product': np.zeros(len(self.products)),
                'interaction': np.zeros(len(list(InteractionType))),
                'rating': 0 
            }
        
        return obs
        
    def _get_interaction_observation(self, interaction:Interaction):
        idx = list(InteractionType).index(interaction.type)
        size = len(InteractionType)
        
        return utils.one_hot_encode(idx, size)
    
    # calculate preferences based on past interactions
    def _get_product_preferences(self, user:Customer):
        view_prefs = user.views / 20
        purchase_prefs = user.buys
        like_prefs = user.likes / 15

        rating_prefs = user.ratings.copy()
        rating_prefs[rating_prefs > 0] -= 2
        
        product_prefs = view_prefs + purchase_prefs + like_prefs+ rating_prefs
        
        product_prefs / 5 # reduce space
        
        return product_prefs    # calculate preferences based on past interactions
    
    def _get_category_preferences(self, user:Customer):
        prod_prefs = self._get_product_preferences(user)
        cat_prefs = np.zeros(len(self.categories), np.float32)
        
        for idx, prod_pref in enumerate(prod_prefs):
            if prod_pref > 0:
                product = self.products[idx]
                cat_idx = product.category.idx
                cat_prefs[cat_idx] += prod_pref # accumulation of fav products for this cat
                # print(f"added pf {prod_pref} to cat {cat_idx}")
                
        cat_prefs = cat_prefs / 5 # reduce space     
           
        return cat_prefs

    def _simulate_interaction(self, product_ids):        
        user = self.users[self.user_idx]
        product_list = []
        
        # simulate selection
        num_products = len(product_ids)
        prod_scores = np.zeros(num_products, np.uint8)
        product_prefs = self._get_product_preferences(user)
        category_prefs = self._get_category_preferences(user)
        product_probs = np.full((num_products,), 1.0 / num_products) # equal probs by default
        
        for idx, pid in enumerate(product_ids):
            product_list.append(self.products[pid]) # get the product objects
            prod_scores[idx] = product_prefs[pid]
            
        # combining category prefs to calculate probabilities
        for idx, product in enumerate(product_list):
            cid = product.category.idx
            prod_scores[idx] += category_prefs[cid] # scores higher for favourite categories
    
        # Ensure the probabilities sum to 1 for a valid probability distribution
        if np.max(prod_scores) > 0: # probability selection based on prefernces
            product_probs = np.array(prod_scores) / sum(prod_scores)

        # Randomly select a product based on the defined probabilities
        selected_product_id = np.random.choice(product_ids, p=product_probs)
        
        # simulate interaction for the selected product
        inter_types = list(InteractionType)
        inter_scores = np.zeros(len(inter_types), np.uint8)
        inter_probs = np.full((len(inter_types),), 1.0 / len(inter_types)) # equal probs by default        
        inter_probs[-1] = 0.1 # lower prob for SESSION_CLOSE ending epsidoe flag to encourage longer training
        
        for idx, inter_type in enumerate(inter_types):
            if inter_type == InteractionType.VIEW:
                inter_scores[idx] = user.views[selected_product_id]
            if inter_type == InteractionType.LIKE:
                inter_scores[idx] = user.likes[selected_product_id]
            if inter_type == InteractionType.BUY:
                inter_scores[idx] = user.buys[selected_product_id]
            if inter_type == InteractionType.RATE:
                inter_scores[idx] = user.ratings[selected_product_id]
        
        if np.max(inter_scores) > 0:
            inter_scores += 1 # default score for interaction that are 0
            inter_scores = inter_scores * inter_probs # scale distribution
            inter_probs = np.array(inter_scores) / sum(inter_scores)

        # Randomly select a product based on the defined probabilities
        inter_probs = np.array(inter_probs) / sum(inter_probs) # normalise
        selected_interaction_type = np.random.choice(inter_types, p=inter_probs)
        
        selected_product_rating = random.randint(0, 5)
        if selected_interaction_type == InteractionType.RATE:
            # likely rating if the product is a prefered product
            prod_pref_score = product_prefs[selected_product_id]
            max_pref_score = np.max(product_prefs)
            if prod_pref_score > 0: # one of favourite products, rating of 2-5
                selected_product_rating = int((prod_pref_score/max_pref_score) * 3) + 2
                
        rating_probs = np.full(6, 1/6)
        rating_probs[selected_product_rating] = 0.7 # up chance of raing
        rating_probs = rating_probs/sum(rating_probs) # normlise
        selected_product_rating = np.random.choice([0,1,2,3,4,5], p=rating_probs)
            
        user_id = self.user_idx
        interaction_time = datetime.now()        
        interaction = Interaction(-1, interaction_time, user_id, selected_product_id, selected_interaction_type, selected_product_rating)

        return interaction

    def seed(self, seed=None):
        """
        Set the seed for reproducibility.
        """
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        random.seed(seed)
        np.random.seed(seed)
        return [seed]
    
    def render(self, mode='human'):
        if hasattr(self, 'last_action'):
            print(f"Recommended Product ID (Last Action): {self.last_action}")
        else:
            print("No product recommended yet.")

        # Optionally, print the reward received for the last action
        if hasattr(self, 'last_reward'):
            print(f"Reward for Last Action: {self.last_reward}")

        print("-----")

    def close(self):
        pass

In [ ]:
products = loader.load_products()
customers = loader.load_customers(include_interactions=True)
categories = loader.load_categories()

In [ ]:
len(products)

In [ ]:
env = RecommendationEnv(customers, products, categories, top_k=10)
env.seed(100)

### Hyperparameter tuning

In [ ]:
param_clip_range = 0.2
param_learning_rate = 0.0007
param_batch_size = 64
param_gamma=0.95
param_gae_lambda=0.95
param_n_steps=100
param_n_epochs=15
param_ent_coef = 0.01
param_total_timesteps = 2000000
save_interval = param_total_timesteps/10

### Initialise model

In [12]:
# Initialize the PPO agent with specified parameters
model = PPO(
    env=env,
    policy='MultiInputPolicy',
    verbose=0,
    clip_range=param_clip_range,
    learning_rate=param_learning_rate,
    n_epochs=param_n_epochs,
    n_steps=param_n_steps,
    ent_coef=param_ent_coef,
    batch_size=param_batch_size,
    gamma=param_gamma,
    gae_lambda=param_gae_lambda,
    # policy_kwargs={'features_extractor_class': CustomANN},
    tensorboard_log=logs_dir
)

/opt/conda/lib/python3.10/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 100`, after every 1 untruncated mini-batches, there will be a truncated mini-batch of size 36
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=100 and n_envs=1)
  warnings.warn(


## Model training

In [ ]:
# Define the interval at which models are saved during training
save_callback = SaveOnIntervalCallback(save_interval, models_dir)

model.learn(total_timesteps=param_total_timesteps, progress_bar=True, callback=save_callback)

# Save the final model after training completes
final_model_path = os.path.join(models_dir, model_name_final)

   7% ━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335,593/5,000,000  [ 1:49:59 < 25:27:53 , 51 it/s ]
   7% ━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335,593/5,000,000  [ 1:49:59 < 25:27:53 , 51 it/s ]

In [ ]:
model.save(final_model_path)

## Testing

In [ ]:
model =  PPO.load(os.path.join(models_dir,'ppo_model_final.zip'))

In [30]:
# Construct observation data for customer 10
test_customer = customers[10] 
test_customer.views[200] = 3
# simulated laset selected product
test_product = products[100]

# it only uses the previous interaction to predict
previous_interaction = Interaction(idx = '0', 
                          timestamp = datetime.now(), 
                          customer_idx = test_customer.idx, 
                          product_idx = test_product.idx, 
                          type = InteractionType.VIEW,
                          value = 5,)

new_obs = env.update_observation(test_customer, previous_interaction)

In [ ]:
new_obs['views'][100]

In [40]:
recommended_products, _states = model.predict(new_obs, deterministic=True)

In [41]:
print(recommended_products)

[250 154 152 274 113 232 273 114 159  61]


In [55]:
new_interaction = Interaction(idx = -1, 
                          timestamp = datetime.now(), 
                          customer_idx = test_customer.idx, 
                          product_idx = test_product.idx, 
                          type = InteractionType.LIKE,
                          value = 3,)

In [56]:
obs, rewards, done, flag, interaction_info = env.step(recommended_products, new_interaction)

In [57]:
recommended_products, _states = model.predict(obs, deterministic=True)

In [58]:
print(recommended_products)

[250 252 152 274 113 232 273  43 159  61]


In [45]:
interaction_info

{'interaction': {'idx': -1,
  'timestamp': '2024-11-07 10:17:25',
  'customer_id': 10,
  'product_id': 100,
  'interaction': 'like',
  'rating': 3}}